In [108]:
import numpy as np
from numba import jit
from scipy.signal import convolve2d
from scipy.sparse import coo_matrix

k = 5


@jit()
def generate_A(N):
    h = 1 / N
    size = (N - 1) ** 2
    const = h ** 2 * k ** 2
    v1 = (24 - 4 * const) / 9
    v2 = (-3 - const) / 9
    v3 = (-12 - const) / 36

    nnz = ((N - 2) * 3 + 1) ** 2
    row = np.zeros(nnz, dtype=np.int32)
    col = np.zeros(nnz, dtype=np.int32)
    data = np.zeros(nnz)

    count = 0
    for i in range(size):
        for j in range(i, size):
            if i == j:
                row[count] = i
                col[count] = j
                data[count] = v1
                count += 1
            elif (i == j - 1 and j % (N - 1) != 0) or j - i == N - 1:
                row[count:count + 2] = [i, j]
                col[count:count + 2] = [j, i]
                data[count:count + 2] = [v2, v2]
                count += 2
            elif (j - i == N and j % (N - 1) != 0) or (j - i == N - 2 and i % (N - 1) != 0):
                row[count:count + 2] = [i, j]
                col[count:count + 2] = [j, i]
                data[count:count + 2] = [v3, v3]
                count += 2
    A = coo_matrix((data, (row, col)), shape=(size, size)).tocsr()
    return A


@jit(nopython=True)
def generate_b(N):
    def func_g(x, y):
        return np.sin(3 * x + 4 * y)

    h = 1 / N
    # nnz = 4 * (N - 2)
    size = (N - 1) ** 2
    data = np.zeros(size)

    const = h ** 2 * k ** 2
    const1 = (12 + const) / 36
    const2 = (3 + const) / 9

    for condition in range(4):
        for position in range(1, N - 1):
            if condition == 0:
                if position == 1:
                    g1 = func_g(0, 0) + func_g(2 * h, 0) + func_g(0, 2 * h)
                    g2 = func_g(h, 0) + func_g(0, h)
                else:
                    g1 = func_g(position * h + h, 0) + func_g(position * h - h, 0)
                    g2 = func_g(position * h, 0)
                row_index = position - 1
            elif condition == 1:
                if position == 1:
                    g1 = func_g(1, 0) + func_g(1, 2 * h) + func_g(1 - 2 * h, 0)
                    g2 = func_g(1 - h, 0) + func_g(1, h)
                else:
                    g1 = func_g(1, position * h + h) + func_g(1, position * h - h)
                    g2 = func_g(1, position * h)
                row_index = position * (N - 1) - 1
            elif condition == 2:
                if position == 1:
                    g1 = func_g(1, 1) + func_g(1 - 2 * h, 1) + func_g(1, 1 - 2 * h)
                    g2 = func_g(1 - h, 1) + func_g(1, 1 - h)
                else:
                    g1 = func_g(position * h + h, 1) + func_g(position * h - h, 1)
                    g2 = func_g(position * h, 1)
                row_index = (N - 1) ** 2 - position
            else:
                position += 1
                if position == N - 1:
                    g1 = func_g(0, 1) + func_g(2 * h, 1) + func_g(0, 1 - 2 * h)
                    g2 = func_g(h, 1) + func_g(0, 1 - h)
                else:
                    g1 = func_g(0, position * h + h) + func_g(0, position * h - h)
                    g2 = func_g(0, position * h)
                row_index = (N - 1) * (position - 1)

            data[row_index] = g1 * const1 + g2 * const2

    return data


def generate_b2(N):
    h = 1 / N
    const = h ** 2 * k ** 2
    const1 = (12 + const) / 36
    const2 = (3 + const) / 9

    window = np.array([[const1, const2, const1],
                       [const2, 0, const2],
                       [const1, const2, const1]])
    x = np.linspace(0, 1, N + 1)
    X, Y = np.meshgrid(x, x)
    X[1:N, 1:N] = 0
    Y = X.view()
    Z = np.sin(3 * X + 4 * Y)
    return convolve2d(Z, window, mode='valid').flatten()

In [109]:
%timeit generate_b2(4)
%timeit generate_b(4)

61.2 µs ± 574 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
The slowest run took 9.27 times longer than the fastest. This could mean that an intermediate result is being cached.
2.49 µs ± 3.15 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
h = .5
N = 2
const = h ** 2 * k ** 2
const1 = (12 + const) / 36
const2 = (3 + const) / 9
g1 = func_g(0, 0) + func_g(1, 1) + func_g(0, 1) + func_g(1, 0)
g2 = func_g(.5, 1) + func_g(.5, 0) + func_g(1, .5) + func_g(0, .5)
data = g1 * const1 + g2 * const2

* solve the matrix-vector problem for $N=4$, $N=8$, and $N=16$

* plot the approximate solutions to the Helmholtz problem.

In [ ]:
def solve_x(N):
    A = generate_A(N)
    b = generate_b(N)
    u = spsolve(A, b).reshape((N - 1, N - 1))
    return u

In [ ]:
% matplotlib inline
from matplotlib import cm

Ns = [4, 8, 16, 55, 100]


def plot():
    fig = plt.figure(figsize=(5 * len(Ns), 22))

    for i, N in enumerate(tqdm(Ns)):
        u = solve_x(N)

        ticksX = ticksY = np.linspace(0, 1, N - 1)
        X, Y = np.meshgrid(ticksX, ticksY)

        ax = fig.add_subplot(1, len(Ns), i + 1, projection='3d')
        ax.plot_surface(X, Y, u, antialiased=False, cmap=cm.coolwarm)
        ax.set_xlabel(r'$x$')
        ax.set_ylabel(r'$y$')
        ax.set_title(r'N = {}'.format(N))

    fig.tight_layout(h_pad=4)
    plt.show()


plot()

* solve the matrix-vector problem with small to medium-sized value of $N$ using a range of different solvers of your choice

    * at least **5** matrix-vector solvers:
        * at least **2** of these should be **iterative** solvers, at least **1** should be a **direct** solver.
        * use some solvers from the Scipy library and some from **PETSc**.

* measuring factors you deem to be important for your evaluation.

    * factors should include the **time** taken by the solver,

        * may additionally include many other things, such as:

          1. the number of **iterations**
          2. the size of the **residual** after each iteration


* Make a set of plots that show the measurements you have made and allow you to compare the solvers.



spsolve
PETSc!
numpy的解法？或者PETSc多选几个
iterative: cg gmres

In [ ]:
!pip install petsc4py

In [ ]:
% matplotlib inline


Ns = np.linspace(2, 500, 50, dtype=np.int32)


class Solver:
    def __init__(self, name: str):
        self.name = name
        self.times = []
        self.solution = []
        self.PC = 'none'
        self.residuals = []
        self.iterations = []


def iterate_pet_solvers(solvers, A, b):
    AA = PETSc.Mat()
    AA.createAIJWithArrays(A.shape, (A.indptr, A.indices, A.data))
    bb = AA.createVecLeft()
    bb.array[:] = b

    for pet_solver in solvers:
        ksp = PETSc.KSP().create()
        ksp.setOperators(AA)
        pet_solver.solution.append(AA.createVecRight())

        ksp.setType(pet_solver.name)
        ksp.getPC().setType(pet_solver.PC)
        ksp.setConvergenceHistory()

        pet_solver.times += [timeit.Timer(lambda: ksp.solve(bb, pet_solver.solution[-1])).timeit(1)]
        pet_solver.residuals += [ksp.getConvergenceHistory()]
        pet_solver.iterations += [ksp.getIterationNumber()]


def test_solvers():
    PET_solvers = [Solver(name) for name in ['dgmres', 'cg', 'cr', 'nash', 'richardson']]
    sp = Solver('spsolver')
    for N in tqdm(Ns):
        A = generate_A(N)
        b = generate_b(N)

        sp.solution += [spsolve(A, b)]
        sp.times += [timeit.Timer(lambda: spsolve(A, b)).timeit(1)]

        iterate_pet_solvers(PET_solvers, A, b)

    PET_solvers.append(sp)

    fig = plt.figure(figsize=(10, 20))

    plt.subplot(3, 1, 1)
    ps = []
    for i, sv in enumerate(PET_solvers):
        p0, = plt.plot(Ns, sv.times)
        ps += [p0]

    plt.legend(ps, [sv.name for sv in PET_solvers], loc='upper left')
    plt.title('Time / N')
    plt.ylabel('time')
    plt.xlabel('N')

    plt.subplot(3, 1, 2)
    ps = []
    for i, sv in enumerate(PET_solvers):
        if i == len(PET_solvers) - 1: continue

        p0, = plt.semilogy(sv.residuals[-1])
        ps += [p0]
    plt.legend(ps, [sv.name for sv in PET_solvers], loc='upper left')
    plt.title('residuals / iterations when N = {}'.format(Ns[-1]))
    plt.ylabel("residuals")
    plt.xlabel('iteration')

    plt.subplot(3, 1, 3)
    ps = []
    for i, sv in enumerate(PET_solvers):
        if i == len(PET_solvers) - 1: continue

        p0, = plt.semilogy(Ns, sv.iterations)
        ps += [p0]
    plt.legend(ps, [sv.name for sv in PET_solvers], loc='upper left')
    plt.title('num of iterations / N')
    plt.ylabel("num of iterations")
    plt.xlabel('N')

    plt.show()


test_solvers()


* For **2** of the iterative solvers you have chosen to use, repeat the comparisons with **3** different choices of **preconditioner**.

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import timeit
from petsc4py import PETSc
from scipy.sparse.linalg import spsolve
from tqdm.notebook import tqdm

Ns = np.linspace(2, 500, 10, dtype=np.int32)


def test_PC():
    PET_solvers = []

    for name in ['cg', 'nash']:
        for PC in ['jacobi', 'sor', 'gasm', 'lu']:
            solver = Solver(name)
            solver.PC = PC
            PET_solvers += [solver]

    for N in tqdm(Ns):
        A = generate_A(N)
        b = generate_b(N)

        iterate_pet_solvers(PET_solvers, A, b)

    fig = plt.figure(figsize=(10, 20))

    plt.subplot(3, 1, 1)
    ps = []
    for i, sv in enumerate(PET_solvers):
        p0, = plt.plot(Ns, sv.times)
        ps += [p0]

    plt.legend(ps, ['{} ({})'.format(sv.name, sv.PC) for sv in PET_solvers], loc='upper left')
    plt.title('Time / N')
    plt.ylabel('time')
    plt.xlabel('N')

    plt.subplot(3, 1, 2)
    ps = []
    for i, sv in enumerate(PET_solvers):
        p0, = plt.semilogy(sv.residuals[-1])
        ps += [p0]

    plt.legend(ps, ['{} ({})'.format(sv.name, sv.PC) for sv in PET_solvers], loc='upper left')
    plt.title('residuals / iterations when N = {}'.format(Ns[-1]))
    plt.ylabel("residuals")
    plt.xlabel('iteration')

    plt.subplot(3, 1, 3)
    ps = []
    for i, sv in enumerate(PET_solvers):
        p0, = plt.semilogy(Ns, sv.iterations)
        ps += [p0]

    plt.legend(ps, ['{} ({})'.format(sv.name, sv.PC) for sv in PET_solvers], loc='upper left')
    plt.title('num of iterations / N')
    plt.ylabel("num of iterations")
    plt.xlabel('N')

    plt.show()


test_PC()


In [35]:
import numpy as np


def get_true_u(N):
    def g(x, y):
        return np.sin(3 * x + 4 * y)

    x = y = np.linspace(0, 1, N)
    X, Y = np.meshgrid(x, y)
    Z = g(X, Y)

    return Z


def calculate_error(u, N):
    error_mat = u - get_true_u(N - 1)



SyntaxError: expected ':' (1544949617.py, line 5)